In [2]:
import random
import math
import numpy as np
import cv2

import gym
from gym import Env, spaces

In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [4]:
########a toi de tester???

In [5]:
import stable_baselines3

In [6]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import os

In [7]:

######################## environement a adapter pour fonctioner sur ce model

In [8]:
BLUE = [255, 0, 0]
#GREEN = [0, 255, 0]
RED = [0, 0, 255]

WHITE = [255, 255, 255]
GREY = [240, 240, 240]
#BLACK = [0, 0, 0]

In [9]:

class City(Env):
    def __init__(self):
        super(City, self).__init__()
        
        self.observation_shape = 10, 10
        self.observation_space = spaces.Box(low = 0, high = 1,
            shape = self.observation_shape, dtype = np.uint8)
        
        self.canvas_shape = 700, 700, 3#10, 10, 3 # width, height, color (BGR)
        self.canvas = np.ones(self.canvas_shape, dtype = np.uint8) * 0
        
        self.action_space = spaces.Discrete(self.observation_shape[0] * self.observation_shape[1],)
        
        self.map = np.ones(self.observation_shape, dtype = np.uint8) * 0
        self.offices = []
        self.houses = []
        pass
    
    def reset(self,random_start = True,start_shape = (3, 3)):
        self.map = np.ones(self.observation_shape, dtype = np.uint8) * 0
        self.offices = []
        self.houses = []
        self.c=0
        if random_start :            
            for y in range((self.observation_shape[1] - start_shape[1]) // 2, (self.observation_shape[1] + start_shape[1]) // 2):
                for x in range((self.observation_shape[0] - start_shape[0]) // 2, (self.observation_shape[0] + start_shape[0]) // 2):
                    self.map[y, x] = random.randrange(3)
                    if self.map[y, x] == 1 : self.offices.append((x, y))
                    elif self.map[y, x] == 2 : self.houses.append((x, y))
        return self.map    
    def __search_nearest_office(self, x, y):
        return int(1)
    
    def step(self, action):
        self.c=self.c+1
        action_x, action_y = action % self.observation_shape[1], action // self.observation_shape[1]
        T=False
        if self.map[action_y, action_x]!=0:
            T=True
            
        
        
        self.houses.append((action_x, action_y))
        self.map[action_y, action_x] = 2
        self.draw_elements_on_canvas()
        #print('juyhbuyhb')

        
        #reward = self.__search_nearest_office(action_x, action_y)
        reward = 100#-reward + 5
        
        #print(self.map)
        
        if T==True:
            reward= -100
        
        print(reward)
        self.render()
        
        res=False
        if self.c>30:
            res=True
            
        return self.map,reward,res,{}
    
    def __draw_element_on_canvas(self, x, y, color):
        observation_width, observation_height = self.observation_shape
        canvas_width, canvas_height, _ = self.canvas_shape

        drawing_width = int(canvas_width / observation_width)
        drawing_height = int(canvas_height / observation_height)

        # fit element to the canvas
        for j in range(y * drawing_height, y * drawing_height + drawing_height):
            for i in range(x * drawing_width, x * drawing_width + drawing_width):
                try : self.canvas[i, j] = color
                except IndexError : pass
                
        for j in range(y * drawing_height, y * drawing_height + drawing_height):
            try : self.canvas[x * drawing_width, j] = GREY
            except IndexError : pass
            
        for i in range(x * drawing_width, x * drawing_width + drawing_width):
            try : self.canvas[i, y * drawing_height] = GREY
            except IndexError : pass
        pass

    def draw_elements_on_canvas(self):
        
        # draw each element of the matrix
        for y in range(len(self.map)):
            for x in range(len(self.map[0])):
                color = WHITE
                if self.map[y, x] == 1 : color = BLUE # offices
                elif self.map[y, x] == 2 : color = RED # houses
                
                self.__draw_element_on_canvas(x, y, color)
        pass
    
    def render(self, mode = "human"):
        if mode == "human" :
            cv2.imshow("", self.canvas)
            cv2.waitKey(10)
            #cv2.destroyAllWindows
        pass

In [10]:
'''env = City()
env.reset()

for _ in range(10):
    env.step(env.action_space.sample())
    env.render()
cv2.waitKey(0)
cv2.destroyAllWindows'''

'env = City()\nenv.reset()\n\nfor _ in range(10):\n    env.step(env.action_space.sample())\n    env.render()\ncv2.waitKey(0)\ncv2.destroyAllWindows'

In [ ]:
import tensorflow as tf
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from tensorflow.keras.layers import Conv2D
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import datasets, layers, models

env = City()
env.observation_space.sample()

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Reshape((10,10,1), input_shape=(1,10,10)))
    model.add(layers.Conv2D(10, (3,3), activation='relu', input_shape=(10, 10,1)))#(1,10, 10)))
    #model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(30, (3,3), activation='relu'))
    #model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(100, (3,3), activation='relu'))
    #model.add(layers.MaxPooling2D((2,2)))

    #model.add(layers.Conv2D(4, (3,3), activation='relu'))
    #model.add(layers.MaxPooling2D((2,2)))
    #model.add(layers.Conv2D(10, (3,3), activation='relu', input_shape=(1,10,10),data_format="channels_first"))#(1,10, 10)))
    #model.add(layers.MaxPooling2D((2,2),data_format="channels_first"))
    #model.add(layers.Conv2D(5, (3,3), activation='relu',data_format="channels_first"))
    #model.add(layers.MaxPooling2D((2,2),data_format="channels_first"))
    #model.add(Dense(24, activation='relu', input_shape=((1,10,10))))
    #model.add(Dense(24, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(states, actions)
from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
def build_agent(model, actions):
    policy = EpsGreedyQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1) 
cv2.waitKey(0)
cv2.destroyAllWindows

Training for 5000 steps ...
Interval 1 (0 steps performed)
100
   98/10000 [..............................] - ETA: 1:25:30 - reward: -61.2245-100


  197/10000 [..............................] - ETA: 1:26:00 - reward: -59.3909100


  296/10000 [..............................] - ETA: 1:24:56 - reward: -56.0811-100


  394/10000 [>.............................] - ETA: 1:23:59 - reward: -61.4213-100


  492/10000 [>.............................] - ETA: 1:23:16 - reward: -63.0081100


...

 4478/10000 [============>.................] - ETA: 48:24 - reward: -40.2858-100


 4579/10000 [============>.................] - ETA: 47:31 - reward: -40.5984100


 4680/10000 [=============>................] - ETA: 46:37 - reward: -41.0256100


 4781/10000 [=============>................] - ETA: 45:49 - reward: -40.8910-100


 4878/10000 [=============>................] - ETA: 45:00 - reward: -41.0004

In [71]:
print(model.summary())

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_14 (Reshape)         (None, 10, 10, 1)         0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 8, 8, 10)          100       
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 6, 6, 30)          2730      
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 4, 4, 100)         27100     
_________________________________________________________________
flatten_15 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               160100    
Total params: 190,030
Trainable params: 190,030
Non-trainable params: 0
_______________________________________________

In [ ]:
# Set up the testing environment

env = City()
env.reset()

# evaluate the agent on a few episodes, drawing randomly from the test samples
dqn.test(env, nb_episodes=1, visualize=True)

cv2.waitKey(0)
cv2.destroyAllWindows

In [ ]:
#entrainer longtemps, revoir la structure